In [0]:
import requests
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession


BASE = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata"

def fetch_data_and_create_dataframe(url_base, date_param_name, date_param_value):
    """
    Função para buscar os dados de um único período e retornar um DataFrame do PySpark.
    """
    all_data = []
    initial_url = f"{url_base}({date_param_name}=@{date_param_name})?@{date_param_name}='{date_param_value}'&$format=json"
    url = initial_url

    while url:
        try:
            r = requests.get(url)
            r.raise_for_status()
            response_json = r.json()
            
            data = response_json.get('value', [])
            all_data.extend(data)
            
            url = response_json.get('@odata.nextLink')
            
            if url:
                print(f"Buscando próxima página para {date_param_value}...")
        
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição para {date_param_value}: {e}")
            return None

    if not all_data:
        print(f"Nenhum dado encontrado para {date_param_value}. Pulando...")
        return None


    df_transacs = pd.DataFrame(all_data)
    

    df_spark_transacs = spark.createDataFrame(df_transacs)
    
    return df_spark_transacs


print("Iniciando coleta e processamento incremental de Estatísticas de Transações Pix...")


for year in range(2023, 2025):
    for month in range(1, 13):
        month_str = f"{year}{month:02d}"
        print(f"\nColetando dados para o mês: {month_str}")


        df_spark_transacs = fetch_data_and_create_dataframe(f"{BASE}/EstatisticasTransacoesPix", "Database", month_str)

        if df_spark_transacs:
            print(f"Dados para {month_str} coletados com sucesso.")
            
            df_spark_transacs.write.format("delta").mode("append").saveAsTable("estatisticas_pix.raw_data.estatisticas_transacoes_pix")
            
            print(f"Dados de estatísticas de transações para {month_str} salvos com sucesso na tabela `estatisticas_pix.raw_data.estatisticas_transacoes_pix`.")

print("\nProcesso ETL incremental concluído.")